In [ ]:
# import needed libraries
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
# SAMPLE URL OF UC DAVIS TO SEE WHAT LINK LOOKS LIKE
url = "https://www.ratemyprofessors.com/search/professors/1073?q=*"


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0.0.0 Safari/537.36",
}
response = requests.get(url, headers=headers)


soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

<html>
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="https://www.ratemyprofessors.com/build/thumbnail.svg" name="thumbnail"/>
  <link href="/manifest.json" rel="manifest"/>
  <script async="" id="nobid-wrapper" src="https://public.servenobid.com/partner/66897/56514/wrapup_1.0.2.js">
  </script>
  <script>
   window.wrapup = window.wrapup || { cmd: [] };
  </script>
  <!-- BounceX / Wunderkind iframebuster -->
  <script charset="utf-8" data-website-id="3905" id="bx-tag" src="//assets.bounceexchange.com/assets/bounce/iframebuster.js">
  </script>
  <!-- GTM -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
      new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
      j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
      'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
      })(window,do

In [ ]:
# look up all urls for UCs

url_sd = "https://www.ratemyprofessors.com/search/professors/1079?q=*"
url_davis = "https://www.ratemyprofessors.com/search/professors/1073?q=*"
url_berkeley = "https://www.ratemyprofessors.com/search/professors/1072?q=*"
url_ucla = "https://www.ratemyprofessors.com/search/professors/1075?q=*"
url_irvine = "https://www.ratemyprofessors.com/search/professors/1074?q=*"
url_sb = "https://www.ratemyprofessors.com/search/professors/1077?q=*"
url_riverside = "https://www.ratemyprofessors.com/search/professors/1076?q=*"
url_merced = "https://www.ratemyprofessors.com/search/professors/4767?q=*"
url_santa_cruz = "https://www.ratemyprofessors.com/search/professors/1078?q=*"

school_ids = ["1079", "1073", "1072", "1075", "1074", "1077", "1076", "4767", "1078"]
base_url = "https://www.ratemyprofessors.com/search/professors/"
complete_urls = [f"{base_url}{school_id}?q=*" for school_id in school_ids]

In [ ]:
### FINAL SCRIPT ### DONT CHANGE ANYTHING
import time
import uuid
import os
import math
import hashlib
import pandas as pd
import re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_rmp(url, school_name, max_clicks=50):
    # Create a unique user data directory for this session
    unique_dir = f"/tmp/chrome-user-data-{uuid.uuid4().hex}"
    os.makedirs(unique_dir, exist_ok=True)

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"--user-data-dir={unique_dir}")

    driver = webdriver.Chrome(options=options)
    driver.get(url)

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//h1[@data-testid='pagination-header-main-results']"))
        )
    except Exception as e:
        print("Initial page load error:", e)

    pbar = tqdm(total=max_clicks, unit="clicks", desc=f"{school_name}: Clicking 'Show More'")
    clicks = 0
    while clicks < max_clicks:
        try:
            btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Show More')]"))
            )
            # Use JavaScript click to speed things up.
            driver.execute_script("arguments[0].click();", btn)
            clicks += 1
            pbar.update(1)
            time.sleep(0.5)  # Reduced sleep time
        except Exception as e:
            print(f"No more 'Show More' button for {school_name} or wait timed out:", e)
            break
    pbar.close()

    # Scrape professor cards.
    cards = driver.find_elements(By.CSS_SELECTOR, "a.TeacherCard__StyledTeacherCard-syjs0d-0")
    data = []
    for card in cards:
        try:
            name_elem = card.find_element(By.CSS_SELECTOR, "div.CardName__StyledCardName-sc-1gyrgim-0")
            full_name = name_elem.text.strip()
            anonymized_name = hashlib.sha256(full_name.encode()).hexdigest()[:8]

            rating_elem = card.find_element(By.CSS_SELECTOR, "div.CardNumRating__CardNumRatingNumber-sc-17t4b9u-2")
            avg_rating = rating_elem.text.strip()

            count_elem = card.find_element(By.CSS_SELECTOR, "div.CardNumRating__CardNumRatingCount-sc-17t4b9u-3")
            num_ratings = count_elem.text.replace("ratings", "").strip()

            dept_elem = card.find_element(By.CSS_SELECTOR, "div.CardSchool__Department-sc-19lmz2k-0")
            department = dept_elem.text.strip()

            feedback_items = card.find_elements(By.CSS_SELECTOR, "div.CardFeedback__CardFeedbackItem-lq6nix-1")
            would_take_again = None
            level_of_difficulty = None
            if len(feedback_items) >= 2:
                wta_div = feedback_items[0].find_element(By.CSS_SELECTOR, "div.CardFeedback__CardFeedbackNumber-lq6nix-2")
                would_take_again = wta_div.text.strip()
                lod_div = feedback_items[1].find_element(By.CSS_SELECTOR, "div.CardFeedback__CardFeedbackNumber-lq6nix-2")
                level_of_difficulty = lod_div.text.strip()

            data.append({
                "Professor": anonymized_name,
                "Average Rating": avg_rating,
                "Mode Rating": avg_rating,
                "Department": department,
                "Would Take Again": would_take_again,
                "Level of Difficulty": level_of_difficulty,
                "Num Ratings": num_ratings,
                "School": school_name
            })
        except Exception as e:
            print(f"Skipping a card in {school_name} due to error:", e)

    driver.quit()
    return pd.DataFrame(data)

# Define school IDs, URLs, and school names.
school_ids = ["1079", "1073", "1072", "1075", "1074", "1077", "1076", "4767", "1078"]
school_names = {
    "1079": "UC San Diego",
    "1073": "UC Davis",
    "1072": "UC Berkeley",
    "1075": "UCLA",
    "1074": "UC Irvine",
    "1077": "UC Santa Barbara",
    "1076": "UC Riverside",
    "4767": "UC Merced",
    "1078": "UC Santa Cruz"
}
base_url = "https://www.ratemyprofessors.com/search/professors/"
complete_urls = {sid: f"{base_url}{sid}?q=*" for sid in school_ids}

dfs = []
for sid, url in complete_urls.items():
    school_name = school_names.get(sid, f"School {sid}")
    print(f"Scraping data for {school_name} from {url}")
    df_school = scrape_rmp(url, school_name, max_clicks=50)
    print(f"Scraped {len(df_school)} records for {school_name}")
    dfs.append(df_school)

df_all = pd.concat(dfs, ignore_index=True)
print(f"Total records scraped: {len(df_all)}")
print(df_all.head(15))


Scraping data for UC San Diego from https://www.ratemyprofessors.com/search/professors/1079?q=*




UC San Diego: Clicking 'Show More':   0%|          | 0/50 [00:00<?, ?clicks/s]

UC San Diego: Clicking 'Show More':   2%|▏         | 1/50 [00:00<00:15,  3.08clicks/s]

UC San Diego: Clicking 'Show More':   4%|▍         | 2/50 [00:02<01:04,  1.33s/clicks]

UC San Diego: Clicking 'Show More':   6%|▌         | 3/50 [00:03<00:58,  1.25s/clicks]

UC San Diego: Clicking 'Show More':   8%|▊         | 4/50 [00:05<01:08,  1.48s/clicks]

UC San Diego: Clicking 'Show More':  10%|█         | 5/50 [00:06<01:06,  1.47s/clicks]

UC San Diego: Clicking 'Show More':  12%|█▏        | 6/50 [00:08<01:02,  1.41s/clicks]

UC San Diego: Clicking 'Show More':  14%|█▍        | 7/50 [00:09<01:03,  1.48s/clicks]

UC San Diego: Clicking 'Show More':  16%|█▌        | 8/50 [00:11<00:59,  1.42s/clicks]

UC San Diego: Clicking 'Show More':  18%|█▊        | 9/50 [00:12<00:53,  1.30s/clicks]

UC San Diego: Clicking 'Show More':  20%|██        | 10/50 [00:13<00:57,  1.44s/clicks]

UC San Diego: Clicking 'Show More':  

Scraped 408 records for UC San Diego
Scraping data for UC Davis from https://www.ratemyprofessors.com/search/professors/1073?q=*




UC Davis: Clicking 'Show More':   0%|          | 0/50 [00:00<?, ?clicks/s]

UC Davis: Clicking 'Show More':   2%|▏         | 1/50 [00:08<07:08,  8.75s/clicks]

UC Davis: Clicking 'Show More':   4%|▍         | 2/50 [00:10<03:32,  4.42s/clicks]

UC Davis: Clicking 'Show More':   6%|▌         | 3/50 [00:10<02:06,  2.69s/clicks]

UC Davis: Clicking 'Show More':   8%|▊         | 4/50 [00:11<01:28,  1.93s/clicks]

UC Davis: Clicking 'Show More':  10%|█         | 5/50 [00:12<01:08,  1.51s/clicks]

UC Davis: Clicking 'Show More':  12%|█▏        | 6/50 [00:12<00:53,  1.21s/clicks]

UC Davis: Clicking 'Show More':  14%|█▍        | 7/50 [00:13<00:44,  1.04s/clicks]

UC Davis: Clicking 'Show More':  16%|█▌        | 8/50 [00:14<00:38,  1.10clicks/s]

UC Davis: Clicking 'Show More':  18%|█▊        | 9/50 [00:14<00:33,  1.24clicks/s]

UC Davis: Clicking 'Show More':  20%|██        | 10/50 [00:15<00:30,  1.31clicks/s]

UC Davis: Clicking 'Show More':  22%|██▏       | 11/50 [00:16<00:28,  1.36clicks/

Scraped 408 records for UC Davis
Scraping data for UC Berkeley from https://www.ratemyprofessors.com/search/professors/1072?q=*


...

Scraped 408 records for UC Riverside
Scraping data for UC Merced from https://www.ratemyprofessors.com/search/professors/4767?q=*




UC Merced: Clicking 'Show More':   0%|          | 0/50 [00:00<?, ?clicks/s]

UC Merced: Clicking 'Show More':   2%|▏         | 1/50 [00:01<00:51,  1.06s/clicks]

UC Merced: Clicking 'Show More':   4%|▍         | 2/50 [00:04<01:53,  2.37s/clicks]

UC Merced: Clicking 'Show More':   6%|▌         | 3/50 [00:04<01:12,  1.55s/clicks]

UC Merced: Clicking 'Show More':   8%|▊         | 4/50 [00:05<00:53,  1.16s/clicks]

UC Merced: Clicking 'Show More':  10%|█         | 5/50 [00:06<00:42,  1.06clicks/s]

UC Merced: Clicking 'Show More':  12%|█▏        | 6/50 [00:06<00:35,  1.23clicks/s]

UC Merced: Clicking 'Show More':  14%|█▍        | 7/50 [00:07<00:31,  1.36clicks/s]

UC Merced: Clicking 'Show More':  16%|█▌        | 8/50 [00:07<00:28,  1.47clicks/s]

UC Merced: Clicking 'Show More':  18%|█▊        | 9/50 [00:08<00:26,  1.55clicks/s]

UC Merced: Clicking 'Show More':  20%|██        | 10/50 [00:08<00:25,  1.55clicks/s]

UC Merced: Clicking 'Show More':  22%|██▏       | 11/50 [00:10<00:32, 

Scraped 408 records for UC Merced
Scraping data for UC Santa Cruz from https://www.ratemyprofessors.com/search/professors/1078?q=*




UC Santa Cruz: Clicking 'Show More':   0%|          | 0/50 [00:00<?, ?clicks/s]

UC Santa Cruz: Clicking 'Show More':   2%|▏         | 1/50 [00:00<00:22,  2.18clicks/s]

UC Santa Cruz: Clicking 'Show More':   4%|▍         | 2/50 [00:05<02:19,  2.90s/clicks]

UC Santa Cruz: Clicking 'Show More':   6%|▌         | 3/50 [00:05<01:29,  1.90s/clicks]

UC Santa Cruz: Clicking 'Show More':   8%|▊         | 4/50 [00:06<01:09,  1.50s/clicks]

UC Santa Cruz: Clicking 'Show More':  10%|█         | 5/50 [00:07<00:53,  1.18s/clicks]

UC Santa Cruz: Clicking 'Show More':  12%|█▏        | 6/50 [00:07<00:43,  1.00clicks/s]

UC Santa Cruz: Clicking 'Show More':  14%|█▍        | 7/50 [00:08<00:39,  1.10clicks/s]

UC Santa Cruz: Clicking 'Show More':  16%|█▌        | 8/50 [00:09<00:37,  1.11clicks/s]

UC Santa Cruz: Clicking 'Show More':  18%|█▊        | 9/50 [00:10<00:34,  1.18clicks/s]

UC Santa Cruz: Clicking 'Show More':  20%|██        | 10/50 [00:10<00:32,  1.23clicks/s]

UC Santa Cruz: Clicking 'S

Scraped 408 records for UC Santa Cruz
Total records scraped: 3672
   Professor Average Rating Mode Rating           Department Would Take Again  \
0   d0c6667d            3.9         3.9              History              67%   
1   c8daa400            1.9         1.9           Management              14%   
2   a528d6a0            2.8         2.8          Mathematics              45%   
3   eedadd80            3.9         3.9              Biology              81%   
4   312bda39            4.2         4.2  Biological Sciences              80%   
5   32cfb9ba            4.2         4.2           Psychology              80%   
6   00fca387            4.2         4.2    Cognitive Science              83%   
7   7652efc1            2.8         2.8  Biological Sciences              45%   
8   8ff910da            4.9         4.9    Cognitive Science              99%   
9   46a51d2c            3.8         3.8             Business              69%   
10  78faf0ee            4.6         4.6    

In [ ]:
df_all

,Professor,Average Rating,Mode Rating,Department,Would Take Again,Level of Difficulty,Num Ratings,School
0,d0c6667d,3.9,3.9,History,67%,3.3,238,UC San Diego
1,c8daa400,1.9,1.9,Management,14%,3.8,234,UC San Diego
2,a528d6a0,2.8,2.8,Mathematics,45%,4.1,216,UC San Diego
3,eedadd80,3.9,3.9,Biology,81%,3,164,UC San Diego
4,312bda39,4.2,4.2,Biological Sciences,80%,3.1,135,UC San Diego
...,...,...,...,...,...,...,...,...
3667,0fd23e3d,2.5,2.5,Humanities,50%,4,2,UC Santa Cruz
3668,5f942b50,4.5,4.5,Anthropology,100%,2.5,2,UC Santa Cruz
3669,f24135df,4.5,4.5,Environmental Studies,100%,3.5,2,UC Santa Cruz
3670,90e5ac76,4.2,4.2,Science,96%,3.2,30,UC Santa Cruz


In [ ]:
df_all.to_csv("uc_rmp_professors.csv", index=False)

In [ ]:
from google.colab import files
files.download("uc_rmp_professors.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>